# Geração de instâncias

In [6]:
from udgp import generate_random_instance

N = 5
instance = generate_random_instance(N)
instance.view()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
from udgp import solve_M4

atoms, time = solve_M4(N, instance.distances)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Set parameter NonConvex to value 2
Set parameter SolutionLimit to value 1
Resolvendo modelo...
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Optimize a model with 673 rows, 275 columns and 1933 nonzeros
Model fingerprint: 0x720f46df
Model has 10 quadratic constraints
Variable types: 175 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 8e+00]
Presolve removed 117 rows and 16 columns
Presolve t

(array([[-0.        , -0.        , -0.        ],
        [-1.13780812, -2.16635878,  0.10466524],
        [-0.97000641,  0.46017481, -1.04753363],
        [-2.91102613, -1.67764107,  1.72192383],
        [-2.11959798, -1.11130874,  0.52054755]]),
 0.22238516807556152)